In [5]:
import psi4
import multiprocessing
from rdkit import Chem
from rdkit.Chem import AllChem
from openbabel import pybel
from concurrent.futures import ProcessPoolExecutor

# ---------- Utility Functions ----------

def rdkit_to_xyz(mol):
    conf = mol.GetConformer()
    n_atoms = mol.GetNumAtoms()
    lines = [str(n_atoms), "Generated by RDKit"]
    for i in range(n_atoms):
        atom = mol.GetAtomWithIdx(i)
        pos = conf.GetAtomPosition(i)
        lines.append(f"{atom.GetSymbol():<2} {pos.x:.6f} {pos.y:.6f} {pos.z:.6f}")
    return "\n".join(lines[2:])  # Return ONLY the atom block (omit count/comment)

def smiles_to_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, randomSeed=42)
    AllChem.UFFOptimizeMolecule(mol)
    return mol

def break_bond(mol, bond_index):
    rwmol = Chem.RWMol(mol)
    bond = rwmol.GetBondWithIdx(bond_index)
    a1, a2 = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
    rwmol.RemoveBond(a1, a2)
    fragments = Chem.GetMolFrags(rwmol, asMols=True, sanitizeFrags=True)
    return fragments

def compute_energy_xyz(xyz_block, charge=0, multiplicity=1):
    psi4.set_options({'reference': 'uhf', 'basis': '6-31G(d)'})
    mol_str = f"""
    {charge} {multiplicity}
    {xyz_block}
    """
    mol = psi4.geometry(mol_str)
    return psi4.energy('B3LYP')

# ---------- Parallel Task Function ----------

def compute_bde_for_bond(args):
    smiles, bond_index = args
    psi4.set_memory('1 GB')
    psi4.core.set_output_file(f'psi4_output_{bond_index}.dat', False)

    try:
        mol = smiles_to_mol(smiles)
        xyz = rdkit_to_xyz(mol)
        E_molecule = compute_energy_xyz(xyz, charge=0, multiplicity=1)

        bond = mol.GetBondWithIdx(bond_index)
        atom1 = bond.GetBeginAtom().GetSymbol()
        atom2 = bond.GetEndAtom().GetSymbol()

        fragments = break_bond(mol, bond_index)
        if len(fragments) != 2:
            return (f"{atom1}-{atom2} (bond {bond_index})", None)

        E_fragments = 0
        for frag in fragments:
            xyz_frag = rdkit_to_xyz(frag)
            E_fragments += compute_energy_xyz(xyz_frag, charge=0, multiplicity=2)

        hartree_to_kcal = 627.509
        bde = (E_fragments - E_molecule) * hartree_to_kcal
        return (f"{atom1}-{atom2} (bond {bond_index})", round(bde, 2))

    except Exception as e:
        return (f"bond {bond_index}", f"Error: {e}")

# ---------- Main BDE Calculator ----------

def compute_all_bdes_serial(smiles):
    mol = smiles_to_mol(smiles)
    bond_indices = [
        bond.GetIdx() for bond in mol.GetBonds()
        if bond.GetBondType() == Chem.BondType.SINGLE
    ]

    bdes = {}
    for idx in bond_indices:
        bond_desc, bde = compute_bde_for_bond((smiles, idx))
        bdes[bond_desc] = bde

    return bdes

def compute_all_bdes_parallel(smiles, max_workers=None):
    mol = smiles_to_mol(smiles)
    bond_indices = [
        bond.GetIdx() for bond in mol.GetBonds()
        if bond.GetBondType() == Chem.BondType.SINGLE
    ]

    args_list = [(smiles, idx) for idx in bond_indices]

    bdes = {}
    with ProcessPoolExecutor(max_workers=max_workers or multiprocessing.cpu_count()) as executor:
        for bond_desc, bde in executor.map(compute_bde_for_bond, args_list):
            bdes[bond_desc] = bde

    return bdes

In [6]:
bdes = compute_all_bdes_serial("CCO")  # Ethanol
for bond_desc, bde in bdes.items():
    print(f"{bond_desc}: {bde} kcal/mol")


  Memory set to 953.674 MiB by Python driver.
C-C (bond 0): 100.61 kcal/mol
C-O (bond 1): 102.55 kcal/mol
C-H (bond 2): 115.42 kcal/mol
C-H (bond 3): 115.02 kcal/mol
C-H (bond 4): 116.48 kcal/mol
C-H (bond 5): 105.18 kcal/mol
C-H (bond 6): 110.46 kcal/mol
O-H (bond 7): 102.69 kcal/mol
